In [1]:
import pandas as pd
raw_dir = "/mnt/c/Users/zohai/Documents/Research/Neuro_ML/LVOs_19_20_Zohair_Aaron/CTAP_19_20_LVO_Test_Data_Final.xls"
raw_df=pd.read_excel(io=raw_dir, sheet_name="Completed")
# print(raw_df.columns)
raw_df.head()
#Excel sheet - 2
#raw_df["RAPID Core"][7]

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


,Organization,Point of Care,Accession Number,Modality,Exam Code,Exam Description,CPT Code,Report Text,LVO,Side,...,EVT,EVT Occlusion,EVT Report,TICI Score,Ordered By,Ordered Date,Exam Completed Date,Report Created Date,Report Finalized By,Report Finalized Date
0,BJH,BJH 0105,55427372,CTA,IMG575,CTA/CTP RAPID STROKE (C),70496,EXAMINATION: Computed tomography angiography (...,0,NaN,...,False,NaN,NaN,NaN,"LEVINE, MARK D.",2020-09-20 16:51:13,2020-09-20 16:25:00,2020-09-20 17:24:56,"Eldaya, Rami W",2020-09-20 17:51:50
1,BJH,BJH 0082,55427150,CTA,IMG575,CTA/CTP RAPID STROKE (C),70496,EXAMINATION: Computed tomography angiography (...,0,NaN,...,False,NaN,NaN,NaN,"KOLLEF, MARIN H.",2020-09-20 15:24:17,2020-09-20 15:15:00,2020-09-20 15:28:59,"Goyal, Manu Shri",2020-09-20 19:32:25
2,BJH,BJH 94ICU,55427201,CTA,IMG575,CTA/CTP RAPID STROKE (C),70496,EXAMINATION: Computed tomography angiography (...,0,NaN,...,False,NaN,NaN,NaN,"GRUBB, LAURIE",2020-09-20 14:59:48,2020-09-20 14:50:00,2020-09-20 15:10:57,"Goyal, Manu Shri",2020-09-20 20:04:06
3,BJH,BJH 94ICU,55426263,CTA,IMG575,CTA/CTP RAPID STROKE (C),70496,EXAMINATION: Computed tomography angiography (...,1,R/L,...,False,NaN,NaN,NaN,"FREER, JAMES MATTHEW",2020-09-20 02:53:03,2020-09-20 02:35:00,2020-09-20 03:05:39,"Goyal, Manu Shri",2020-09-20 09:56:07
4,BJH,BJH 0082,55408274,CTA,IMG575,CTA/CTP RAPID STROKE (C),70496,EXAMINATION: Computed tomography angiography (...,0,NaN,...,False,NaN,NaN,NaN,"LOVELESS, REBECCA ANNE",2020-09-17 03:05:10,2020-09-17 02:20:00,2020-09-17 03:06:56,"Vo, Katie D.",2020-09-17 07:33:09


In [2]:
# trim_df = raw_df[["LVO","Side","site occlusion","CTA CTP Report Text","RAPID Core","RAPID Penumbra"]]
# trim_df.head()

In [3]:
trim_df = raw_df[["LVO","Side","site occlusion","Report Text"]]
trim_df.head()

,LVO,Side,site occlusion,Report Text
0,0,NaN,NaN,EXAMINATION: Computed tomography angiography (...
1,0,NaN,NaN,EXAMINATION: Computed tomography angiography (...
2,0,NaN,NaN,EXAMINATION: Computed tomography angiography (...
3,1,R/L,M1/Vertebral,EXAMINATION: Computed tomography angiography (...
4,0,NaN,NaN,EXAMINATION: Computed tomography angiography (...


In [4]:
#Uncertain LVO 1?
# trim_df.iloc[504]

In [5]:
#Filter if LVO uncertain
# filtered_df = trim_df[~trim_df["LVO"].str.contains("?",regex=False).fillna(False)]
# filtered_df.iloc[50]
filtered_df = trim_df

In [6]:
#Function that reads the angio report cerebral artery table
#from pandas dataframe
#and outputs a python dictionary
#
#If "no occlusion" is present for a given artery, value is marked as 0
#Otherwise, the program looks to free-body text for clarification
#These values are marked with 'None'
import copy
import re

class ca_templates_reader:
    
    def __init__(self,df_column):
        
        self.df_column = df_column
        
        #cerebral arteries dictionary
        self.main_dict = {
            "L CCA": None,
            "L carotid bifurcation": None,
            "L ICA proximal": None,
            "L ICA distal": None,
            "L ICA terminus": None,
            "L M1": None,
            "L M2 branches": None,
            "L A2": None,
            "R CCA": None,
            "R carotid bifurcation": None,
            "R ICA proximal": None,
            "R ICA distal": None,
            "R ICA terminus": None,
            "R M1": None,
            "R M2 branches": None,
            "R A2": None,
            "L Vertebral Artery": None,
            "R Vertebral Artery": None,
            "Basilar Artery": None,
            "L PCA": None,
            "R PCA": None

        }
        
        self.ca_pattern_dict = copy.deepcopy(self.main_dict)
    
    def compile_regex(self):
        #Compiling regex patterns of the cerebral artery strings
        #For optimal performance
        
        for ca in self.ca_pattern_dict:
            self.ca_pattern_dict[ca]=re.compile(ca+':.*\n')
        
    
    def read_ca_template(self, text):
        
        #Example cerebral arteries template line:
        #R CCA: No occlusion or significant stenosis

        no_occlusion = re.compile('(n|N)o occlusion')
    #     for file in raw_text_files:
        curr_dict = copy.deepcopy(self.main_dict)
        for ca in curr_dict:
            #Searching for template text for each cerebral artery
            #Some follow up reports may not have the template
            
            search = self.ca_pattern_dict[ca].search(text)
            if search is not None:
                #string of ca template line
                sent = search.group()
                #print(sent)
                #Expect match most cases, change dict value to 0 (no occlusion)
                #If 'no occlusion' not in cerebral a. desc, leave with None value (default flag)
                #Will determine later
                if no_occlusion.search(sent) is not None:
                    curr_dict[ca] = 0
        return curr_dict
    #         ca_dicts.append(curr_dict)
    
    def read_templates(self):
        #List to store the cerebral artery dictionaries for report
        ca_dicts = []
        for report in self.df_column:
            ca_dicts.append(self.read_ca_template(report))
        return ca_dicts

In [7]:
ctr = ca_templates_reader(filtered_df["Report Text"])
ctr.compile_regex()

In [8]:
class report_text_extractor:
    
    def __init__(self, df_column):
        self.df_column = df_column
        self.report_template_pattern = re.compile('Left anterior circulation:[\s\S]*IMPRESSION', re.I)
        #List of all cerebral artery templates from the reports
        self.report_templates = self.extract_from_reports(self.report_template_pattern)


        self.impression_pattern = re.compile('IMPRESSION:[\s\S]*', re.I)
        #List of all impressions from the reports
        self.impressions = self.extract_from_reports(self.impression_pattern)
        
        #List of all CT Perfusion sections from reports
        self.ctp_pattern = re.compile('CT Perfusion:[\s\S]*IMPRESSION', re.I)
        self.ctp_sections = self.extract_from_reports(self.ctp_pattern)
        
        
    def get_report_templates(self):
        return self.report_templates
    
    def get_impressions(self):
        return self.impressions

    def get_ctps(self):
        return self.ctp_sections
    
    def extract_from_reports(self,pattern):
        report_extractions = []
        for report in self.df_column:
            search = pattern.search(report)
            if search:
                report_extractions.append(search.group())
            else:
                report_extractions.append(None)
        return report_extractions


In [9]:
rte = report_text_extractor(filtered_df["Report Text"])
templates = rte.get_report_templates()
impressions = rte.get_impressions()
ctps = rte.get_ctps()
print(templates[300])
print('/'*50)
print(impressions[300])
print('/'*50)
#Extract ischemic core volume and hypoperfusion values in mLprint(ctps[522])

Left anterior circulation:
L CCA: no occlusion or significant stenosis
L carotid bifurcation: no occlusion or significant stenosis
L ICA proximal: no occlusion or significant stenosis
L ICA distal: no occlusion or significant stenosis
L ICA terminus: no occlusion or significant stenosis
L M1: no occlusion or significant stenosis
L M2 branches: no occlusion or significant stenosis
L A2: no occlusion or significant stenosis

Right anterior circulation:
R CCA: no occlusion or significant stenosis
R carotid bifurcation: no occlusion or significant stenosis
R ICA proximal: no occlusion or significant stenosis
R ICA distal: no occlusion or significant stenosis
R ICA terminus: no occlusion or significant stenosis
R M1: no occlusion or significant stenosis
R M2 branches: no occlusion or significant stenosis
R A2: no occlusion or significant stenosis
 
Posterior circulation:
L Vertebral Artery: no occlusion or significant stenosis
R Vertebral Artery: no occlusion or significant stenosis
Basilar

In [10]:
#Extract ischemic core and hypoperfusion volumes in mL
class ctp_volumes_extractor:
    def __init__(self,ctp_sections_list):
        self.ctps = ctp_sections_list
        self.volume_pattern = re.compile('[0-9]* *m[Ll]')
        self.volumes = []
        
    def extract_volumes(self):
        for ctp in self.ctps:
            if ctp:
                #List of 2 strings ex. ['0 mL', '146 mL']
                search = self.volume_pattern.findall(ctp)
                if search:                      
                    #['0 mL', '146 mL'] -> (0, 146)
#                     vol_tup = tuple([int(vol_text.split()[0]) if len(vol_text.split()) > 1 else None
#                                      for vol_text in search])
                    vol_tup = tuple([int(vol) for vol_text in search for vol in re.findall(r'\d+',vol_text)])
                    self.volumes.append(vol_tup)
                else:
                    self.volumes.append(None)
            else:
                self.volumes.append(None)

    #List of tuples of format (ischemic core volume,hypoperfusion volume)
    def get_volumes(self):
        return self.volumes
        
        

In [11]:
ctp_str = '''
CT Perfusion:

Estimated ischemic core volume (rCBF < 0.3): 177ml
Estimated hypoperfusion volume (Tmax > 6 sec): 320 mL


IMPRESSION:'''
test_ctp = [ctp_str]
ctpve = ctp_volumes_extractor(test_ctp)
ctpve.extract_volumes()
ctpve.get_volumes()

[(177, 320)]

In [12]:
print(filtered_df["Report Text"].iloc[172])

EXAMINATION: Computed tomography angiography (CTA) of the head without and with contrast; Computed tomography angiography (CTA) of the neck with contrast.
CT perfusion imaging of the head with contrast

HISTORY: Suspected hyperacute stroke.

TECHNIQUE: Computed tomography of the head was performed without contrast according to standard protocol. Computed tomographic angiography was obtained from the level of the aortic arch to the vertex following the uneventful administration of intravenous contrast according to hyperacute stroke protocol. 3D images were generated on a dedicated workstation.
CT perfusion of the brain was performed with intravenous contrast using a separate data acquisition. The data was transmitted to a separate workstation for processing by RAPID software (iSchemaView) to produce automated calculations of the estimated cerebral blood flow and Tmax.

Contrast information:
125 mL Optiray-350

COMPARISON: CT head dated 5/18/2020 and 2/5/2020.

FINDINGS:

HEAD CT FINDING

In [13]:
ctr.read_ca_template(templates[8])


{'L CCA': 0,
 'L carotid bifurcation': None,
 'L ICA proximal': 0,
 'L ICA distal': None,
 'L ICA terminus': 0,
 'L M1': 0,
 'L M2 branches': 0,
 'L A2': 0,
 'R CCA': 0,
 'R carotid bifurcation': None,
 'R ICA proximal': 0,
 'R ICA distal': None,
 'R ICA terminus': 0,
 'R M1': 0,
 'R M2 branches': 0,
 'R A2': 0,
 'L Vertebral Artery': 0,
 'R Vertebral Artery': 0,
 'Basilar Artery': 0,
 'L PCA': 0,
 'R PCA': 0}

In [14]:
ca_templates_dicts = ctr.read_templates()
ca_templates_dicts[52]

{'L CCA': 0,
 'L carotid bifurcation': None,
 'L ICA proximal': 0,
 'L ICA distal': None,
 'L ICA terminus': None,
 'L M1': 0,
 'L M2 branches': 0,
 'L A2': 0,
 'R CCA': 0,
 'R carotid bifurcation': None,
 'R ICA proximal': 0,
 'R ICA distal': None,
 'R ICA terminus': None,
 'R M1': 0,
 'R M2 branches': 0,
 'R A2': 0,
 'L Vertebral Artery': 0,
 'R Vertebral Artery': 0,
 'Basilar Artery': None,
 'L PCA': 0,
 'R PCA': 0}

In [15]:
#For empty values in the ca_dicts, need to check impression
#NLP needs to be used here ex.'Occlusion of the proximal inferior left M2 branch'
#NLP needed on midline shift ex. There is no mass effect or midline shift.
import networkx as nx
import pyConTextNLP.pyConText as pyConText
import pyConTextNLP.itemData as itemData

In [16]:
modifiers = itemData.get_items("/mnt/c/Users/zohai/Documents/Research/Neuro_ML/Sample_Data/modifiers.yml")
targets = itemData.get_items("/mnt/c/Users/zohai/Documents/Research/Neuro_ML/Sample_Data/targets.yml")

/mnt/c/Users/zohai/Documents/Research/Neuro_ML/miniconda3/envs/neuroml/lib/python3.8/site-packages/pyConTextNLP/itemData.py:40: YAMLLoadWarning: calling yaml.load_all() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  context_items =  [contextItem((d["Lex"],


In [17]:
#pyConTextNLP uses yml files to store entities and their regex pattern information
#This function is to be used for quick lookup of an entity for short term use
#Or to add new entities to a yml file

#"/mnt/c/Users/zohai/Documents/Research/Neuro_ML/Sample_Data/modifiers.yml"
#"/mnt/c/Users/zohai/Documents/Research/Neuro_ML/Sample_Data/targets.yml"

class pyConText_itemData_initializer:
    
    def __init__(self, modifiers_path=None, targets_path=None):
        if modifiers_path is None:
            self.modifiers_path = "https://raw.githubusercontent.com/chapmanbe/pyConTextNLP/master/KB/lexical_kb_05042016.yml"
        else:
            self.modifiers_path = modifiers_path
            
        self.modifiers = itemData.get_items(self.modifiers_path)
        self.targets = []
            
        if targets_path is not None:
            self.targets_path = targets_path
            self.targets = itemData.get_items(self.targets_path)
    
    def add_target(self,entity,category='', regex='', direction=''):
        self.targets.append(itemData.contextItem([entity,category,regex,direction]))
        
        #Code from pyConTextNLP itemData.py
        #def get_items(_file): context_items =  [contextItem((d["Lex"], d["Type"],r"%s"%d["Regex"],d["Direction"])) for d in yaml.load_all(f0)]
    
    def add_entity_to_yml(self, yml_path, entity, category="''", regex="''", direction="''"):
        yml_entity = "Direction: "+ direction + "\n" + \
        "Lex: "+ entity + "\n" + \
        "Regex: "+ regex + "\n" + \
        "Type: " + category + "\n" + \
        "---\n" 
        
        with open(yml_path, "r+") as f:
            old = f.read()
            f.seek(0)
            #Prepending
            f.write(yml_entity + old)
            
        f.close()
        
    def get_itemData(self):
        return self.modifiers,self.targets
        


In [18]:
mod_path = "/mnt/c/Users/zohai/Documents/Research/Neuro_ML/Sample_Data/modifiers.yml"
targ_path = "/mnt/c/Users/zohai/Documents/Research/Neuro_ML/Sample_Data/targets.yml"
itemData_init = pyConText_itemData_initializer(mod_path,targ_path)
itemData_init.modifiers[3]

literal<<\bl\b>>; category<<['left_side']>>; re<<\bl\b>>; rule<<forward>>

In [19]:
type(itemData_init.targets[0])

pyConTextNLP.itemData.contextItem

In [20]:
# itemData_init.add_target('occlusion')
itemData_init.targets[0]

literal<<occlusion>>; category<<['occlusion']>>; re<<(occlusion|occlusive|occluded)>>; rule<<bidirectional>>

In [21]:
#itemData_init.add_entity_to_yml("/mnt/c/Users/zohai/Documents/Research/Neuro_ML/Sample_Data/targets.yml",'P1')

In [22]:
#Function to determine existence or rule out given entity
#Ex. Midline shift/No midline shift
#Ex. Occlusion/No occlusion

#Input is string entity that you want to rule in/out
#***Since words have DIFFERENT forms, consider using a PREFIX INSTEAD
#Ex. Entity: occlusion; sentence contains 'occluded thrombus'
# Using the prefix 'occlu' would capture both in a rudimentary fashion

class modifier_target_finder:
    def __init__(self, sentence, mod_itemData, targ_itemData):
        self.modifiers = mod_itemData
        self.targets = targ_itemData
        self.sentence = sentence
        
        self.markup = pyConText.ConTextMarkup()
        self.markup.setRawText(sentence.lower())
        self.markup.cleanText()
        
        self.markup.markItems(self.modifiers,mode="modifier")
        self.markup.markItems(self.targets,mode="target")
        self.markup.applyModifiers()
    
    #New addition
    def log(self):
        print(self.markup.__str__())
        
    
    def get_markup(self):
        return list(self.markup.edges())
            
    def find_mod_target_pair(self,desired_modifier,modifier_attrib,desired_target,target_attrib):
        #set removes duplicates
        return list(set([(modifier.getLiteral(),target.getLiteral()) 
                for (modifier,target) in self.markup.edges() 
                # getattr() will run modifier/target . getLiteral()/categoryString()/getTagID()/getConTextCategory()
                if (desired_modifier in getattr(modifier,modifier_attrib)() 
                    and desired_target in getattr(target,target_attrib)())]))
    
    
    #Returns 0 if negated
    #Returns 1 if present and not negated
    #Returns 2 if confirmed existence
    #Returns None if not in sentence
    def confirm_existence(self, entity, entity_type):
        entities = []
        for modifier,target in self.markup.edges():
            etype_func = {"getLiteral":[modifier.getLiteral,target.getLiteral],
                          "categoryString":[modifier.categoryString,target.categoryString]}
            for func in etype_func[entity_type]:
                entities.append(func())
        if entity in entities:
            neg_pairs_list = self.find_mod_target_pair('negated_existence','categoryString',entity,entity_type)
            if neg_pairs_list:
                return 0
            def_exis_pairs_list = self.find_mod_target_pair(('definite_existence'),
                                                    'categoryString', entity,entity_type)
            prob_exis_pairs_list = self.find_mod_target_pair(('definite_existence'),
                                                            'categoryString', entity,entity_type)
            if def_exis_pairs_list or prob_exis_pairs_list:
                return 2
            return 1
        else:
            return None
    

In [23]:
mtf = modifier_target_finder("Abrupt cut off of distal right M2/ proximal M3 with geographic paucity of vasculature compared to contralateral side. ",
                             itemData_init.get_itemData()[0],itemData_init.get_itemData()[1])
mtf.get_markup()

[(<id> 43921701137426022956509316448840336511 </id> <phrase> right </phrase> <category> ['right_side'] </category> ,
  <id> 43950117110193389003830616922067846271 </id> <phrase> m2 </phrase> <category> ['m2_circulation'] </category> ),
 (<id> 43921701137426022956509316448840336511 </id> <phrase> right </phrase> <category> ['right_side'] </category> ,
  <id> 43950158308837896421286165564922020991 </id> <phrase> m3 </phrase> <category> ['m3_circulation'] </category> ),
 (<id> 43922801616603346088158490774310503551 </id> <phrase> m2 </phrase> <category> ['m2_circulation'] </category> ,
  <id> 43950117110193389003830616922067846271 </id> <phrase> m2 </phrase> <category> ['m2_circulation'] </category> ),
 (<id> 43922801616603346088158490774310503551 </id> <phrase> m2 </phrase> <category> ['m2_circulation'] </category> ,
  <id> 43950158308837896421286165564922020991 </id> <phrase> m3 </phrase> <category> ['m3_circulation'] </category> ),
 (<id> 43922887975300486636286467737216369791 </id> <p

In [24]:
if mtf.find_mod_target_pair("negated_existence","categoryString","uncertain_flow","categoryString"):
    print(True)

In [25]:
mtf.confirm_existence("uncertain_occlusion","categoryString")

1

In [26]:
#Extract midline shift yes/no
#Searches entire report for midline shift sentence,
#as it is discussed at beginning of Findings section
#exclusively
class midline_shift_extractor:
    def __init__(self,df_column,modifiers,targets):
        self.df_column = df_column
        self.modifiers = modifiers
        self.targets = targets
        self.midline_shift_pattern = re.compile('([^.]*?midline shift[^.]*\.)')
        self.midline_shift_statuses = []
        
    def extract_midline_shifts(self):
        for report in self.df_column:
            search = self.midline_shift_pattern.search(report)
            if search:
                sent = search.group()
                mtf = modifier_target_finder(sent,self.modifiers,self.targets)
                status = mtf.confirm_existence("midline shift","getLiteral")
                if status == 0:
                    self.midline_shift_statuses.append(0)
                elif status > 0:
                    self.midline_shift_statuses.append(1)
                else:
                    self.midline_shift_statuses.append(None)
    
    def get_midline_shifts(self):
        return self.midline_shift_statuses
                
            

In [27]:
# mse = midline_shift_extractor(filtered_df["CTA CTP Report Text"],itemData_init.get_itemData()[0],itemData_init.get_itemData()[1])
# mse.extract_midline_shifts()
# midline_shifts = mse.get_midline_shifts()
# midline_shifts

In [28]:
# [i for i, value in enumerate(midline_shifts) if value == 1] 
#Only 2 out of 500 have midline shift, will not run in final function

In [29]:
# filtered_df['CTA CTP Report Text'][211]

In [30]:
#ca = cerebral arteries
#expects ca_templates_reader dictionary as input
#which acts as a filter.
#For sites not ruled out (value=None), read the impression
#for further clarification

class possible_occlusions_generator:    
    def __init__(self, ca_filter_dict):
        
        self.new_dict_keys = ["L CCA", "L ICA", "L ACA", "L A1", "L A2", "L A3", "L A4", "L A5",
                        "L MCA", "L M1", "L M2", "L M3", "L M4",
                        "R CCA", "R ICA", "R ACA", "R A1", "R A2", "R A3", "R A4", "R A5",
                        "R MCA", "R M1", "R M2", "R M3", "R M4",
                        "L vertebral", "L V1", "L V2", "L V3", "L V4",
                        "R vertebral", "R V1", "R V2", "R V3", "R V4",
                        "basilar",
                        "L PCA", "L P1", "L P2", "L P3", "L P4",
                        "R PCA", "R P1", "R P2", "R P3", "R P4"]
    
        
        self.filter_dict = self.consolidate_dict(ca_filter_dict)
        self.filled_dict = self.fill_in_dict(self.filter_dict)
        self.ca_to_investigate = [key for key,value in self.filled_dict.items() if value < 0]
        
    def negate_nones(self,dictionary):
        negated_dict = {}
        for key,value in dictionary.items():
            if value is None:
                negated_dict[key] = -1
            else:
                negated_dict[key] = value
        return negated_dict
            
    def consolidate_keys_for_dict(self,dictionary,list_of_keys,consol_key):
        new_dict = {}
        consolidated_value = 0
        found_key = 0
        for key,value in dictionary.items():
            if found_key == 1:
                #Add consol_key to new_dict at spot of first key in list_of_keys 
                new_dict[consol_key] = consolidated_value
            if key in list_of_keys:
                found_key+=1                
                #Value is negative, indicating further investigation
                consolidated_value += value                
            else:
                new_dict[key] = value
        new_dict[consol_key] = consolidated_value
        return new_dict
    
    #Calls consolidate_keys_for_dict to consolidate CCAs and ICAs
    def consolidate_dict(self,filter_dict):
        consol_dict = self.negate_nones(filter_dict)
        keys_dict = {
            "L CCA": ["L CCA", "L carotid bifurcation"],
            "R CCA": ["R CCA", "R carotid bifurcation"],
            "L ICA": ["L ICA proximal","L ICA distal","L ICA terminus"],
            "R ICA": ["R ICA proximal","R ICA distal","R ICA terminus"],
            "L M2": ["L M2 branches"],
            "R M2": ["R M2 branches"],
            "R vertebral": ["R Vertebral Artery"],
            "L vertebral": ["L Vertebral Artery"],
            "basilar": ["Basilar Artery"]
        }
        for consol_key,list_of_keys in keys_dict.items():
            consol_dict = self.consolidate_keys_for_dict(consol_dict,list_of_keys,consol_key)
        
        return consol_dict
    
    
    #Adds AX, MX, PX, and VX segments, where X is segment location (ex. V4, X=4)
    def fill_in_dict(self,consol_dict):
        
        new_dict = {}
        
        #Populate keys-vals from previous consolidated dictionary
        for key in self.new_dict_keys:
            if key in consol_dict:
                new_dict[key] = consol_dict[key]
            else:
                #Temp value that will be filled by transfer_key_values
                new_dict[key] = None
        
        #The template serves as a reference for which cerebral arteries to look at
        #This function transfers the values from the template to the corresponding
        #AX, MX, PX, VX arteries
        #Ex. L Vertebral Artery Occlusion indicates occlusion in one/more of L V1-4
        def transfer_key_values(source_key,list_receiving):
            for key in list_receiving:
                if key == "L MCA":
                    new_dict[key] = consol_dict["L M1"]+consol_dict["L M2"]
                elif key == "R MCA":
                    new_dict[key] = consol_dict["R M1"]+consol_dict["R M2"]
                else:
                    new_dict[key] = consol_dict[source_key]
        refs = {
            "L M1/M2": ["L MCA"],
            "R M1/M2": ["R MCA"],
            "L M2": ["L M3", "L M4"],
            "R M2": ["R M3", "R M4"],
            "L A2": ["L ACA","L A1", "L A3", "L A4", "L A5"],
            "R A2": ["R ACA","R A1", "R A3", "R A4", "R A5"],
            "L vertebral": ["L V1", "L V2", "L V3", "L V4"],
            "R vertebral": ["R V1", "R V2", "R V3", "R V4"],
            "L PCA": ["L P1", "L P2", "L P3", "L P4"],
            "R PCA": ["R P1", "R P2", "R P3", "R P4"]
        }
        
        for source_key,list_receiving in refs.items():
            transfer_key_values(source_key,list_receiving)
            
        return new_dict
    
    def get_possibilities(self):
        return self.ca_to_investigate

In [31]:
from itertools import compress
class occlusions_investigator:
    
    def __init__(self,ca_to_investigate,impressions_txt,modData,targData):
        self.modData = modData
        self.targData = targData
        self.impressions_txt = impressions_txt
        self.ca_to_investigate = ca_to_investigate
        
        self.extra_chars = re.compile('IMPRESSION:|\n[0-9]\.|\r|\n|\t')
        self.impressions = re.sub(self.extra_chars,'',self.impressions_txt)
        self.txt_sents = self.impressions.split(".")
        #List of 0,1,2 or Nones based on negated occ, occ mentioned, occ confirmed, or no occ mentioned
        self.sent_has_occlusion = [self.confirm_occlusion(sent) for sent in self.txt_sents]
        self.sent_maybe_occlusion = [self.confirm_uncertain_occlusion(sent) for sent in self.txt_sents]
        self.occlusions = []
        self.bool_none = lambda x: False if not x else True
        self.filt_sent_has_occlusion = list(filter(self.bool_none,self.sent_has_occlusion))
        self.filt_txt_sents = list(compress(self.txt_sents,map(self.bool_none,self.sent_has_occlusion)))
        self.filt_maybe_sents = list(compress(self.txt_sents,map(self.bool_none,self.sent_maybe_occlusion)))
        #0 for negation
        #1 for occlusion present and not negated
        #2 for occlusion indicated
        #None for occlusion not found
        self.occlusion_ratings = []
        
        #This list keeps track of occlusion sentences that have and have not
        #been matched with an occlusion in the template
        #Those unmatched with template will be flagged for the user to review
        self.unmatched_occlusion_sents = [True for sent in self.filt_txt_sents]
        
        #Find if sentence refers to cerebral artery with indeterminate terms
        #Will be flagged for user review
        self.uncertain_occlusions = []
        
        self.markups = []

    def get_occlusions(self):
        
        ca_to_investigate = []
        for potential in self.ca_to_investigate:
            side = None
            ca = None
            if " " in potential:
                potential_list = potential.split(" ")
                side = potential_list[0]
                ca = potential_list[1]
            else:
                ca = potential
            
            ca_to_investigate.append((potential,side,ca))
        
        #Check for uncertain occlusions
        if self.sent_maybe_occlusion:
            #print(self.sent_maybe_occlusion)
            for sent in self.filt_maybe_sents:
                for potential,side,ca in ca_to_investigate:
                    lim = self.confirm_limitation(sent, ca)
                    if lim:
                        self.uncertain_occlusions.append(lim)
        
        #There is no occlusion in any sentence
        #Return empty occlusions list
        if not self.filt_sent_has_occlusion:
            self.occlusion_ratings = [None for pos in self.ca_to_investigate]
            return self.occlusions

        #Investigate potenial occlusions by side and cerebral artery
        for potential,side,ca in ca_to_investigate:
            #for sent,occl_label in zip(self.filt_txt_sents,self.filt_sent_has_occlusion):
            for i in range(len(self.filt_txt_sents)):
                sent = self.filt_txt_sents[i]
                occl_label = self.filt_sent_has_occlusion[i]
                #If site/side of occlusion verified
                #Add the potential occlusion to the final occlusions list
                is_occl, flag = self.confirm_site_occlusion(sent,side,ca)
                #Excludes both None and 0, runs on 1 or 2
                if is_occl:
                    self.unmatched_occlusion_sents[i] = False
                    if flag == -1:
                        #side of the site could not be verified, flag -1
                        #in occlusion_ratings list
                        self.occlusion_ratings.append((sent,side,ca,flag))
                        #Marked as non-occlusion for now
                        occl_label = 0
                    else:
                        #Can be corellated to filt_sent_has_occlusion list with labels 0-2
                        self.occlusion_ratings.append((sent,side,ca,occl_label))
            
                    if occl_label > 0:
                        self.occlusions.append(potential)
                else:
                    #is_occl
                    #None: Means potential occlusion not found in Findings
                    #0: Occlusion negated
                    self.occlusion_ratings.append((sent,side,ca,None))
                        
            
        self.unmatched_occlusion_sents = list(compress(self.filt_txt_sents,self.unmatched_occlusion_sents))
        

        #If other side has no findings and occlusion of that artery present
        #Then add to occlusions list
        def check_other_side_occlusion(side,ca,occl_label):
            
            vs_dict = {"ACA": ["A1","A2","A3","A4","A5"],
                       "MCA": ["M1","M2","M3","M4"],
                       "PCA": ["P1","P2","P3","P4"],
                       "vertebral": ["V1","V2","V3","V4"]
                      }
            
            other_side = None
            if side == "L":
                other_side = "R"
            elif side == "R":
                other_side = "L"
            for o_sent, o_side, o_ca, o_occl_label in self.occlusion_ratings:
                #Most cases will be a list of one element
                #Ex. M2
                ca_list = [ca]
                #Need to check other side segment equivalents
                #If general large vessel a possible occlusion
                #Ex. L MCA -> Check R M1, R M2, R M3, R M4
                if ca in vs_dict:
                    for c in vs_dict[ca]:
                        ca_list.append(c)
                for ca_equiv in ca_list:
                    #Check if other side, same cerebral artery and occlusion label is 0,1,2
                    #If so this occlusion is not likely
                    #Otherwise, add this occlusion
                    if o_occl_label is not None:
                        if o_side == other_side and o_ca == ca_equiv and o_occl_label > -1:
                            return
            self.occlusions.append(side + " " + ca)

        #Check flagged occlusions where side and cerebral artery didn't match
        for sent,side,ca,occl_label in self.occlusion_ratings:
            if occl_label == -1:             
                check_other_side_occlusion(side,ca,occl_label)
                

        return self.occlusions
    
    def get_possibility_ratings(self):
        return self.occlusion_ratings
    
    def get_markups(self):
        #(index,sents with occlusion, markup of those sents)
        return list(zip(self.filt_txt_sents,self.markups))
    
    def get_unmatched_occlusion_sents(self):
        return self.unmatched_occlusion_sents
    
    def get_uncertain_occlusions(self):
        return self.uncertain_occlusions


    def confirm_occlusion(self,sent):
        mtf = modifier_target_finder(sent,self.modData,self.targData)
        #Looking for occlusion in acute context, not chronic occlusion
        if not mtf.find_mod_target_pair('chronic','getLiteral','occlusion','getLiteral'):
            #either 0, 1, 2, or None
            return mtf.confirm_existence('occlusion','getLiteral')
        else:
            return None
        
    def confirm_uncertain_occlusion(self,sent):
        mtf = modifier_target_finder(sent,self.modData,self.targData)
        uo = mtf.confirm_existence('uncertain_occlusion','categoryString')
        if uo:
            return uo
        uf = mtf.confirm_existence('uncertain_flow','categoryString')
        if uf:
            return uf
        return None


    def confirm_site_occlusion(self,sent,side,site):
        mtf = modifier_target_finder(sent,self.modData,self.targData)
        self.markups.append(mtf.get_markup())
        full_side = side
        if side == "L":
            full_side = "left_side"
        elif side == "R":
            full_side = "right_side"

        #Verify occlusion of specific cerebral artery
        sites = mtf.find_mod_target_pair(site,'getLiteral','occlusion','getLiteral')
        
        #Verify side of specific cerebral artery
        #None gets overwritten later, flag -1 important
        if sites:
            if side is None:
                return (True,None)
            sides_sites = mtf.find_mod_target_pair(full_side,'categoryString',site,'getLiteral')
            if sides_sites:
                return (True,None)
            else:
                #could not verify the side corresponds to site in findings
                return (True,-1)
        return (False,None)
    
    #Sentences with terms that may or may not suggest acute occlusion
    #Are flagged for user review
    def confirm_limitation(self, sent, site):
        mtf = modifier_target_finder(sent,self.modData,self.targData)
        if mtf.find_mod_target_pair('uncertain_occlusion','categoryString',site,'getLiteral'):
            return ('uncertain occlusion',site,sent)
        if mtf.find_mod_target_pair('uncertain_flow','categoryString',site,'getLiteral'):
            if mtf.find_mod_target_pair('negated_existence','categoryString','uncertain_flow','categoryString'):
                return ('uncertain negated flow',site,sent)
        return None
        
        

In [32]:
u = ('t1','t2')
p = ('t3')
zip('t1','t2','t3')

In [33]:
# occlusions_list = filtered_edges(('anterior_circulation' or 'posterior_circulation'),'categoryString','occlusion','categoryString')
# side_circ_list = []
# #occlusions_list = [('M1', 'occlusion'), ('M2', 'occlusion')]
# for (modifier,target) in occlusions_list:
#     #side_circ_list = [('right', 'M1'), ('right', 'M2')]
#     side_circ_list += filtered_edges('side','categoryString',modifier,'getLiteral')
# for side,circ in side_circ_list:
#     #list evaluates to [('no', 'occlusion')], or is empty
#     no_occlusion = filtered_edges('negated_existence','categoryString','occlusion','categoryString')
#     if no_occlusion:
#         print("No occlusion at "+ side + " " + circ)
#     else:
#         print("Occlusion at "+ side + " " + circ)

# # else:
# #     for (modifier,target) in occlusions_list:
# #             print("occlusion at " + occlusions_list[0])

In [34]:
test_dict = ca_templates_dicts[325]
test_dict

{'L CCA': None,
 'L carotid bifurcation': None,
 'L ICA proximal': None,
 'L ICA distal': None,
 'L ICA terminus': None,
 'L M1': None,
 'L M2 branches': None,
 'L A2': None,
 'R CCA': None,
 'R carotid bifurcation': None,
 'R ICA proximal': None,
 'R ICA distal': None,
 'R ICA terminus': None,
 'R M1': None,
 'R M2 branches': None,
 'R A2': None,
 'L Vertebral Artery': None,
 'R Vertebral Artery': None,
 'Basilar Artery': None,
 'L PCA': None,
 'R PCA': None}

In [35]:
pog = possible_occlusions_generator(test_dict)
pog.filter_dict

{'L CCA': -2,
 'L ICA': -3,
 'L M1': -1,
 'L M2': -1,
 'L A2': -1,
 'R CCA': -2,
 'R ICA': -3,
 'R M1': -1,
 'R M2': -1,
 'R A2': -1,
 'L vertebral': -1,
 'R vertebral': -1,
 'basilar': -1,
 'L PCA': -1,
 'R PCA': -1}

In [36]:
pog.filled_dict

{'L CCA': -2,
 'L ICA': -3,
 'L ACA': -1,
 'L A1': -1,
 'L A2': -1,
 'L A3': -1,
 'L A4': -1,
 'L A5': -1,
 'L MCA': -2,
 'L M1': -1,
 'L M2': -1,
 'L M3': -1,
 'L M4': -1,
 'R CCA': -2,
 'R ICA': -3,
 'R ACA': -1,
 'R A1': -1,
 'R A2': -1,
 'R A3': -1,
 'R A4': -1,
 'R A5': -1,
 'R MCA': -2,
 'R M1': -1,
 'R M2': -1,
 'R M3': -1,
 'R M4': -1,
 'L vertebral': -1,
 'L V1': -1,
 'L V2': -1,
 'L V3': -1,
 'L V4': -1,
 'R vertebral': -1,
 'R V1': -1,
 'R V2': -1,
 'R V3': -1,
 'R V4': -1,
 'basilar': -1,
 'L PCA': -1,
 'L P1': -1,
 'L P2': -1,
 'L P3': -1,
 'L P4': -1,
 'R PCA': -1,
 'R P1': -1,
 'R P2': -1,
 'R P3': -1,
 'R P4': -1}

In [37]:
possibilities = pog.get_possibilities()
possibilities

['L CCA',
 'L ICA',
 'L ACA',
 'L A1',
 'L A2',
 'L A3',
 'L A4',
 'L A5',
 'L MCA',
 'L M1',
 'L M2',
 'L M3',
 'L M4',
 'R CCA',
 'R ICA',
 'R ACA',
 'R A1',
 'R A2',
 'R A3',
 'R A4',
 'R A5',
 'R MCA',
 'R M1',
 'R M2',
 'R M3',
 'R M4',
 'L vertebral',
 'L V1',
 'L V2',
 'L V3',
 'L V4',
 'R vertebral',
 'R V1',
 'R V2',
 'R V3',
 'R V4',
 'basilar',
 'L PCA',
 'L P1',
 'L P2',
 'L P3',
 'L P4',
 'R PCA',
 'R P1',
 'R P2',
 'R P3',
 'R P4']

In [38]:
print(filtered_df["Report Text"].iloc[325])

EXAMINATION: Computed tomography angiography (CTA) of the head without and with contrast
Computed tomography angiography (CTA) of the neck with contrast

HISTORY: Trauma.

TECHNIQUE: Computed tomography of the head was performed without contrast according to standard protocol. Computed tomographic angiography was then obtained from the aortic arch to the vertex following the uneventful administration of intravenous contrast. 3D images were generated on a dedicated workstation.

Contrast information:
100 mL Optiray-350

COMPARISON: Head CT from 12/20/2019. 

FINDINGS:

Topogram demonstrates no lytic lesions or fractures. 

There is interval increase in size of a left convexity subdural hematoma now measuring 2.1 cm, previously 1.2 cm but with decreasing hyperdensity.  There is increasing mass effect with a new 5 mm rightward midline shift.

Ventricles are of normal size and morphology. No mass effect or midline shift is present. The gray-white matter differentiation is normal. The visua

In [39]:
oi = occlusions_investigator(possibilities,impressions[325],itemData_init.get_itemData()[0],itemData_init.get_itemData()[1])
predicted_occlusions = oi.get_occlusions()
predicted_occlusions

[]

In [40]:
#predicted_occlusions vs possibilities
extra_poss = list(set(predicted_occlusions) - set(possibilities))
extra_poss

[]

In [41]:
oi.unmatched_occlusion_sents

[]

In [42]:
oi.uncertain_occlusions

[]

In [43]:
oi.confirm_uncertain_occlusion('cut-off')

In [44]:
oi.confirm_limitation('Abrupt cut off of distal right M2/ proximal M3 with geographic paucity of vasculature compared to contralateral side. ','M2')

('uncertain occlusion',
 'M2',
 'Abrupt cut off of distal right M2/ proximal M3 with geographic paucity of vasculature compared to contralateral side. ')

In [45]:
oi.filt_txt_sents

[]

In [46]:
#oi.occlusion_ratings
oi.get_possibility_ratings()

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [47]:
oi.filt_sent_has_occlusion

[]

In [48]:
oi.sent_has_occlusion

[None,
 None,
 None,
 None,
 0,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [49]:
oi.sent_maybe_occlusion

[None,
 None,
 None,
 None,
 None,
 1,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [50]:
oi.filt_txt_sents

[]

In [51]:
class vessel_site_classifier:
    
    def __init__(self,occl_list):
        self.occl_list = occl_list
        #Large vessels: CCA, ICA, ACA, MCA, PCA, Vertebrals, Basilar
        #Segments (of large vessels):
        #A1-5, M1-4, V1-4, P1-4
        self.vessels = []
        self.sites = []
        self.vs_dict = {"CCA": [],
                       "ICA": [],
                       "ACA": ["A1","A2","A3","A4","A5"],
                       "MCA": ["M1","M2","M3","M4"],
                       "PCA": ["P1","P2","P3","P4"],
                       "vertebral": ["V1","V2","V3","V4"],
                       "basilar": []}
    def classify(self):
        for occlusions in self.occl_list:
            vessel_occlusions = []
            site_occlusions = []
            for occlusion in occlusions:
                if len(occlusion.split()) > 1:
                    side = occlusion.split()[0]
                    occl = occlusion.split()[1]
                else:
                    side = ''
                    occl = occlusion
                if occl in self.vs_dict.keys():
                    vessel_occlusions.append(side + " " + occl)
                else:
                    for vessel,segments in self.vs_dict.items():
                        if occl in segments:
                            vessel_occlusions.append(side + " " + vessel)
                            #Ex ('MCA', 'L')
                            site_occlusions.append(side + " " + occl)
            #list(set()) removes duplicates (ex. multiple MCA occlucions -> ['MCA'])
            self.vessels.append(list(set(vessel_occlusions)))
            self.sites.append(site_occlusions)
        return self.vessels,self.sites
                
            

In [52]:
#Test vessel_site_classifier
test_vsc = vessel_site_classifier([predicted_occlusions])
predicted_vessel_occ, predicted_segments_occ = test_vsc.classify()
lower = lambda x:x.lower()

In [53]:
predicted_vessel_occ

[[]]

In [54]:
predicted_segments_occ

[[]]

In [55]:
list(set(map(lower,predicted_vessel_occ[0])))

[]

In [56]:
l = lambda q:q.split()[-1]
f = [" MCA","R ICA", "L vertebral","L ICA"]
list(set(map(l,f)))
#s.split()[-1]

['vertebral', 'MCA', 'ICA']

In [57]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import accuracy_score
class stats_generator:
    def __init__(self, X, y, check_side=None):
        self.oneHotEncoder = MultiLabelBinarizer()
        
        if check_side:
            #Get the 'L', or 'R' if there is a side, or put "None"
            self.splitter = lambda w:w.split()[0] if ('L' in w) or ('R' in w) else 'None'
        else:
            #Get rid of side labels
            #["L MCA"] -> ["MCA"]
            #***Merges both side occlusions ex. ["R MCA","L MCA"] -> ["MCA"]
            self.splitter = lambda l:l.split()[-1]
        self.X = [list(set(map(self.splitter,i))) for i in X]
        self.y = [list(set(map(self.splitter,i))) for i in y]
        #If only self.y, classes in self.X not in self.y would
        #cause an error when classification_report(...) generated
        #Case where predictions vary from y labels 
        self.oneHotEncoder.fit(self.y+self.X)
        self.encoded_X = self.oneHotEncoder.transform(self.X)
        self.encoded_y = self.oneHotEncoder.transform(self.y)
    def getStats(self):
        print(classification_report(self.encoded_y, self.encoded_X, target_names=self.oneHotEncoder.classes_))
        print(multilabel_confusion_matrix(self.encoded_y, self.encoded_X))
        print(accuracy_score(self.encoded_y, self.encoded_X))
        
        
        
#         self.vessels = ["CCA","ICA","ACA","MCA","PCA","vertebral","basilar"]
#         self.sites = ["A1","A2","A3","A4","A5","M1","M2","M3","M4","P1","P2","P3","P4","V1","V2","V3","V4"]
        
        

In [58]:
testX = [['R basilar'],['L vertebral'],['L MCA','L ICA']]
testy = [['L vertebral', ' basilar', 'R vertebral'],['L vertebral'],['L ICA']]
sg = stats_generator(testX, testy)
print(sg.encoded_X)
print(sg.encoded_y)
sg.oneHotEncoder.classes_

[[0 0 1 0]
 [0 0 0 1]
 [1 1 0 0]]
[[0 0 1 1]
 [0 0 0 1]
 [1 0 0 0]]


array(['ICA', 'MCA', 'basilar', 'vertebral'], dtype=object)

In [59]:
print(classification_report(sg.encoded_y, sg.encoded_X, target_names=sg.oneHotEncoder.classes_))

              precision    recall  f1-score   support

         ICA       1.00      1.00      1.00         1
         MCA       0.00      0.00      0.00         0
     basilar       1.00      1.00      1.00         1
   vertebral       1.00      0.50      0.67         2

   micro avg       0.75      0.75      0.75         4
   macro avg       0.75      0.62      0.67         4
weighted avg       1.00      0.75      0.83         4
 samples avg       0.83      0.83      0.78         4



/mnt/c/Users/zohai/Documents/Research/Neuro_ML/miniconda3/envs/neuroml/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [60]:
# testX = [['R basilar'],['L vertebral'],[]]
# testy = [['L vertebral', ' basilar', 'R vertebral'],['L vertebral'],['R MCA']]
sg = stats_generator(testX, testy, "sides")
print(sg.encoded_X)
print(sg.encoded_y)
sg.oneHotEncoder.classes_

[[0 0 1]
 [1 0 0]
 [1 0 0]]
[[1 1 1]
 [1 0 0]
 [1 0 0]]


array(['L', 'None', 'R'], dtype=object)

In [61]:
print(classification_report(sg.encoded_y, sg.encoded_X, target_names=sg.oneHotEncoder.classes_))

              precision    recall  f1-score   support

           L       1.00      0.67      0.80         3
        None       0.00      0.00      0.00         1
           R       1.00      1.00      1.00         1

   micro avg       1.00      0.60      0.75         5
   macro avg       0.67      0.56      0.60         5
weighted avg       0.80      0.60      0.68         5
 samples avg       1.00      0.78      0.83         5



/mnt/c/Users/zohai/Documents/Research/Neuro_ML/miniconda3/envs/neuroml/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [62]:
class labels_extractor:
    
    def __init__(self,side_column,occlusion_column):
        self.side_labels = self.extract_occlusion_labels(side_column)
        self.occl_labels = self.extract_occlusion_labels(occlusion_column)
    
    def extract_occlusion_labels(self,df):
        #side = (df['Side'])
        cleaned_df = list(df.fillna(''))

        #df_normalize_both = [df.lower() if "both" in df.lower() else df for df in cleaned_df]

        df_no_spaces = [df.replace(" ","") for df in cleaned_df]

        df_no_seps = [re.split("\/|&",df) if ("/" in df or "&" in df) else df for df in df_no_spaces]


        return df_no_seps
    
    def return_labels(self):
        labels = []
        for side, occl in zip(self.side_labels,self.occl_labels):
            #Occl is '', i.e. no occlusion
            if not occl:
                labels.append([])
            else:
                #Lowercase vertebral and basilar to match predictions
                if occl in ["Vertebral", "Basilar"]:
                    occl = occl.lower()
                #Ex. ["M2", "M1"]
                if isinstance(occl,list):
                    #Ex. ["L", "R"]
                    if isinstance(side,list):
                        labels.append([" ".join(tup) for tup in zip(side,occl)])
#                         for s, o in zip(side,occl):
#                             labels.append(s + " " + o)
                    else:
                        if side:
                            side += " "
                        l = []
                        for o in occl:
                            if "both" in side.lower():
                                l.append("R "+ o)
                                l.append("L "+ o)
                            else:
                                l.append(side + o)
                        labels.append(l)
                else:
                    if side:
                        side += " "
                    if "both" in side.lower():
                        labels.append(["R "+ occl, "L "+ occl])
#                         l.append("R "+ occl)
#                         l.append("L "+ occl)
                    else:
                        labels.append([side + occl])
        return labels
                        
                    

    
# side_labels = extract_occlusion_labels(filtered_df['Side'])
# side_labels
le = labels_extractor(filtered_df['Side'],filtered_df['site occlusion'])
le.side_labels[421]

'R'

In [63]:
# occl_labels = extract_occlusion_labels(filtered_df['site occlusion'])
# occl_labels
# le.occl_labels[452]

In [64]:
labels = le.return_labels()
# labels[452]

In [65]:
# labels[452]

In [66]:
predicted_occlusions

[]

In [67]:
predicted_occlusions == labels[129]

True

In [68]:
import numpy as np
class test_occlusions:
    
    def __init__(self,df):
        self.df = df
        self.df_report_column = self.df["Report Text"]
        
        #For output excel file
        self.output_dir = ""
        
        #Class to extract desired portions of reports
        self.report_extractor = report_text_extractor(self.df_report_column)
        #Cerebral artery templates
        #NOT USED
        self.report_ca_templates = self.report_extractor.get_report_templates()
        #End 'Impression:' section
        self.impressions = self.report_extractor.get_impressions()
        #CTP Perfusion section
        self.ctp_sections = self.report_extractor.get_ctps()
        
        self.ctpve = ctp_volumes_extractor(self.ctp_sections)
        self.ctpve.extract_volumes()
        ##List of tuples of format (ischemic core volume,hypoperfusion volume)
        self.ctp_volumes = self.ctpve.get_volumes()
        
        #Goes reads through all templates and populates a dictionary
        self.ca_templates_reader = ca_templates_reader(self.df_report_column)
        self.ca_templates_reader.compile_regex()
        self.ca_templates_dicts = self.ca_templates_reader.read_templates()
        
        #Setting up pyConText
        self.mod_path = "/mnt/c/Users/zohai/Documents/Research/Neuro_ML/Sample_Data/modifiers.yml"
        self.targ_path = "/mnt/c/Users/zohai/Documents/Research/Neuro_ML/Sample_Data/targets.yml"
        self.itemData_init = pyConText_itemData_initializer(self.mod_path,self.targ_path)
        self.modifiers = self.itemData_init.get_itemData()[0]
        self.targets = self.itemData_init.get_itemData()[1]
        
        #Finding possible occlusions based on cerebral arteries templates
        self.ca_to_investigate = []
        self.pogs = []
        for template in self.ca_templates_dicts:
            pog = possible_occlusions_generator(template)
            self.pogs.append(pog)
            self.ca_to_investigate.append(pog.get_possibilities())
        
        #Investigating the possible occlusions of the impressions of each report
        #Using pyConText (used by occlusions_investigator)
        
        
        #self.predicted_occlusions -> Raw occlusions found, used to populate self.occlusion_labels,
        # self.predicted_vessels, and self.predicted_segments
        self.predicted_occlusions = []
        #For debugging, -1 represents flag of uncertainty
        self.possibility_ratings = []
        self.markups = []
        self.unmatched_occlusion_sents = []
        self.uncertain_occlusions = []
        
        for possibilities,impression in zip(self.ca_to_investigate,self.impressions):
            
            
            oi = occlusions_investigator(possibilities,impression,self.modifiers,self.targets)
            self.predicted_occlusions.append(oi.get_occlusions())
            self.possibility_ratings.append(oi.get_possibility_ratings())
            self.markups.append(oi.get_markups())
            self.unmatched_occlusion_sents.append(oi.get_unmatched_occlusion_sents())
            self.uncertain_occlusions.append(oi.get_uncertain_occlusions())
            
            
        self.pred_occl_statuses = [0 if not occl else 1 for occl in self.predicted_occlusions]
        self.occl_status_labels = [1 if val > 0 else val for val in self.df['LVO']]
        
        self.pred_classifier = vessel_site_classifier(self.predicted_occlusions)
        self.predicted_vessels, self.predicted_segments = self.pred_classifier.classify()
        
        #Extracting labels to verify the predicted occlusions
        self.labels_extractor = labels_extractor(self.df['Side'],self.df['site occlusion'])
        self.labels = self.labels_extractor.return_labels()
        
        self.labels_classifier = vessel_site_classifier(self.labels)
        self.labeled_vessels, self.labeled_segments = self.labels_classifier.classify()        
                
        #self.differences = [(prediction,label) if list(set(map(self.lower,prediction))^set(map(self.lower,label))) 
                       #else () for prediction,label in zip(self.predicted_occlusions,self.labels) ]
        self.diffs = self.differences(self.predicted_occlusions,self.labels)
        #Indices of differences of occlusion status
        self.occl_status_diffs = np.where((abs(np.subtract(self.pred_occl_statuses,self.occl_status_labels)).astype(bool)))
        self.vessels_diffs = self.differences(self.predicted_vessels,self.labeled_vessels)
        self.segments_diffs = self.differences(self.predicted_segments,self.labeled_segments)
    

    
    def differences(self,preds,labels):
        #Compare lower case labels to lower case predictions
        #Set ^ operation returns list with elements unique to either prediction or label but not both
        lower = lambda x:x.lower()
        differences = []
        for prediction,label in zip(preds,labels):
            if list(set(map(lower,prediction))^set(map(lower,label))):
                differences.append((prediction,label))
            else:
                differences.append(())
        return differences
    
     
    #An uncertainty flag for user review
    #may show a template error
    #returns the sentences and report indices where occlusions are mentioned in the findings 
    #but the algo did not register any at those same cerebral arteries in template
    def get_unmatched_occlusions(self):
        return [(sent,i) for sent,i in zip(self.unmatched_occlusion_sents,range(len(self.unmatched_occlusion_sents))) if sent]
    
    #For user review of instances where occlusion is possible because of similar terms used, but occlusion not used
    def get_uncertain_occlusions(self):
        return [(uo,i) for uo,i in zip(self.uncertain_occlusions,range(len(self.uncertain_occlusions))) if uo] 
        #return self.uncertain_occlusions
    
    def get_occlusion_metrics(self):
            #Precision/PPV = TP/TP+FP i.e. out of true calls, how many actually true
            #Recall = TP/TP+FN i.e. out of all trues, how many algo called true
            #F1-score = Accuracy taking into account precision and recall
            print(classification_report(to.occl_status_labels,to.pred_occl_statuses))
    
    def get_vessel_metrics(self):
        vessel_sg = stats_generator(self.predicted_vessels,self.labeled_vessels)
        vessel_sg.getStats()
        
    def get_segment_metrics(self):
        segment_sg = stats_generator(self.predicted_segments,self.labeled_segments)
        segment_sg.getStats()
        
    def get_side_metrics(self):
        side_sg = stats_generator(self.predicted_segments,self.labeled_segments,"sides")
        side_sg.getStats()
    
    def get_occlusion_accuracy(self):
        #Subtracts the 1's and 0's of the predicted and labeled occl statuses and turns them positive, sums, divides
        #by the total to get the error rate, then yields accuracy by 1-error_rate
        return(1-(sum(abs(np.subtract(self.pred_occl_statuses,self.occl_status_labels)))/len(self.occl_status_labels)))
    
    def get_vessel_accuracy(self):
        return self.get_accuracy(self.vessels_diffs)
    
    def get_segments_accuracy(self):
        return self.get_accuracy(self.segments_diffs)
    
    def get_raw_accuracy(self):
        return self.get_accuracy(self.diffs)
        
        
    def get_accuracy(self,diffs_list):
        score = 0
        for tup in diffs_list:
            if not tup:
                score += 1
        return(score/len(diffs_list))
    
    def get_volumes(self):
        return self.ctp_volumes
      
    #Writes to an excel file
    #(Takes input excel file, adds columns, and outputs it)
    #Input: Input excel file dataframe, input excel file path
    def write_to_excel(self,raw_df, raw_dir):
        output_df = raw_df
        self.output_dir = raw_dir.split(".")[0] + "_AUTOMATED_OCCLUSION_PREDICTIONS.xlsx"
        outputs = {"pred_occlusion": self.pred_occl_statuses,
                   "pred_vessels": self.predicted_vessels,
                   "pred_segments": self.predicted_segments,
                   "true_vessels": self.labeled_vessels,
                   "true_segments": self.labeled_segments,
                   "volumes": self.get_volumes(),
                   "unmatched_occlusions": self.unmatched_occlusion_sents,
                   "uncertain_occlusions": self.uncertain_occlusions
                  }
        for col_name, col_data in outputs.items():
            output_df[col_name] = col_data
        output_df.to_excel(self.output_dir,index=False)
    
    def get_output_excel_path(self):
        return self.output_dir
            
        
        
            
            
        
        
        
        
        

In [69]:
print(filtered_df["LVO"].iloc[200:210])

200    0
201    0
202    1
203    0
204    0
205    0
206    1
207    1
208    0
209    0
Name: LVO, dtype: int64


In [70]:
q = ('','')
if not q:
    print('not q')

In [71]:
to = test_occlusions(filtered_df)
#print(to.report_ca_templates[90])


/mnt/c/Users/zohai/Documents/Research/Neuro_ML/miniconda3/envs/neuroml/lib/python3.8/site-packages/pyConTextNLP/itemData.py:40: YAMLLoadWarning: calling yaml.load_all() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  context_items =  [contextItem((d["Lex"],


In [72]:
to.pred_occl_statuses

[0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,


In [73]:
to.occl_status_labels

[0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,


In [74]:
to.get_occlusion_metrics()

              precision    recall  f1-score   support

           0       0.95      0.97      0.96       320
           1       0.92      0.85      0.89       122

    accuracy                           0.94       442
   macro avg       0.93      0.91      0.92       442
weighted avg       0.94      0.94      0.94       442



In [75]:
to.get_vessel_metrics()

              precision    recall  f1-score   support

         ACA       0.60      1.00      0.75         3
         CCA       0.33      0.50      0.40         2
         ICA       0.83      0.87      0.85        23
         MCA       0.90      0.84      0.87        86
         PCA       1.00      0.71      0.83         7
     basilar       0.67      1.00      0.80         4
   vertebral       0.47      0.82      0.60        11

   micro avg       0.80      0.84      0.82       136
   macro avg       0.69      0.82      0.73       136
weighted avg       0.84      0.84      0.83       136
 samples avg       0.22      0.23      0.22       136

[[[437   2]
  [  0   3]]

 [[438   2]
  [  1   1]]

 [[415   4]
  [  3  20]]

 [[348   8]
  [ 14  72]]

 [[435   0]
  [  2   5]]

 [[436   2]
  [  0   4]]

 [[421  10]
  [  2   9]]]
0.9027149321266968


/mnt/c/Users/zohai/Documents/Research/Neuro_ML/miniconda3/envs/neuroml/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/mnt/c/Users/zohai/Documents/Research/Neuro_ML/miniconda3/envs/neuroml/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [76]:
to.get_segment_metrics()

              precision    recall  f1-score   support

          A1       0.00      0.00      0.00         0
          A2       0.67      0.67      0.67         3
          M1       0.85      0.80      0.82        44
          M2       0.82      0.84      0.83        43
          M3       0.00      0.00      0.00         3
          M4       0.00      0.00      0.00         0
          P1       1.00      1.00      1.00         1
          P2       0.67      0.67      0.67         3
          P3       1.00      1.00      1.00         1
          V1       0.00      0.00      0.00         0
          V2       0.00      0.00      0.00         0
          V3       0.50      0.25      0.33         4
          V4       0.50      0.50      0.50         8

   micro avg       0.73      0.75      0.74       110
   macro avg       0.46      0.44      0.45       110
weighted avg       0.77      0.75      0.76       110
 samples avg       0.17      0.17      0.17       110

[[[440   2]
  [  0   0]]


In [77]:
to.get_side_metrics()

              precision    recall  f1-score   support

           L       0.86      0.79      0.83        48
           R       0.86      0.81      0.83        52

   micro avg       0.86      0.80      0.83       100
   macro avg       0.86      0.80      0.83       100
weighted avg       0.86      0.80      0.83       100
 samples avg       0.18      0.18      0.18       100

[[[388   6]
  [ 10  38]]

 [[383   7]
  [ 10  42]]]
0.9298642533936652


In [104]:
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(to.occl_status_labels,to.pred_occl_statuses).ravel()
(tn, fp, fn, tp)

(311, 9, 18, 104)

In [105]:
segs = list(zip(to.predicted_segments, to.labeled_segments))
segs

[([], []),
 ([], []),
 ([], []),
 (['R M1'], ['R M1']),
 ([], []),
 ([], ['R M1']),
 ([], []),
 ([], []),
 ([], []),
 (['R P2'], ['R P2']),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], ['R A2']),
 ([], []),
 (['R M2'], ['R M2']),
 ([], []),
 ([], []),
 (['R M1'], ['R M1']),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 (['L M2'], ['L M2']),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 (['L V4', 'R V4'], []),
 ([], []),
 ([], ['L M2']),
 ([], []),
 ([], []),
 ([], []),
 ([], ['R M3']),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 (['R M1', 'R M2'], []),
 (['L V4'], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 (['L V4'], ['L V4']),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 (['R M1'], ['R M1']),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),


In [106]:
o = ['L M1', ' basilar', 'M']
seg_m1 = lambda x: int(not not sum(["M1" in l for l in x]))
seg_m2 = lambda x: int(not not sum(["M2" in l for l in x]))
seg_m2(o)

0

In [107]:
#Only M1 or M2s in segs
segs = [(segs[i][0],segs[i][1]) for i in range(len(segs)) if (seg_m1(segs[i][1]) or seg_m2(segs[i][1]))]
segs

[(['R M1'], ['R M1']),
 ([], ['R M1']),
 (['R M2'], ['R M2']),
 (['R M1'], ['R M1']),
 (['L M2'], ['L M2']),
 ([], ['L M2']),
 (['R M1'], ['R M1']),
 (['R M1'], ['R M1']),
 ([], ['R M1']),
 (['R M2'], ['R M2']),
 (['R M1', 'R M2'], ['R M2']),
 (['L M1', 'L M2'], ['L M2']),
 (['L M1', 'L M2', 'L M4'], ['L M1']),
 (['L M1'], ['L M1']),
 (['L M1'], ['L M1']),
 (['R M2', 'R M2'], ['R M2']),
 (['L M1'], ['L M1']),
 ([], ['R M2']),
 ([], ['R M2', 'R M3']),
 (['L M1'], ['L M1']),
 (['R M1'], ['R M1']),
 (['R M1'], ['R M1']),
 (['R M1', 'R M2'], ['R M1', 'R M2']),
 (['L M2'], ['L M2']),
 (['R M1'], ['R M1']),
 (['L M2'], ['L M2']),
 (['L M2'], ['L M2']),
 (['R M1'], ['R M1']),
 (['L M2'], ['L M2']),
 (['L M1'], ['L M1']),
 (['L M2'], ['L M2']),
 (['L M2'], ['L M2']),
 (['L M2'], ['L M2']),
 (['L M2'], ['L M2']),
 (['L M2', 'R M2'], ['R M2', 'L M2']),
 (['R M2'], ['R M2']),
 (['L M2'], ['L M2']),
 (['L M2'], ['L M2']),
 (['L M1'], ['L M1']),
 (['L M2'], ['L M2']),
 (['R M2'], ['R M2']),
 (['R M

In [108]:
len(segs)

81

In [109]:
seg_m1_status_df = pd.DataFrame([(seg_m1(p),seg_m1(l)) for (p,l) in segs])
seg_m1_status_df.columns = ["predicted","labels"]
seg_m1_status_df

,predicted,labels
0,1,1
1,0,1
2,0,0
3,1,1
4,0,0
...,...,...
76,1,1
77,0,1
78,0,0
79,0,0


In [110]:
# seg_m1_df = pd.DataFrame([(seg_m1(p),seg_m1(l)) for (p,l) in segs])
# seg_m1_df.columns = ["predicted","labels"]
# seg_m2_df = pd.DataFrame([(seg_m2(p),seg_m2(l)) for (p,l) in segs])
# seg_m2_df.columns = ["predicted","labels"]
# seg_m1_df

In [111]:
# seg_m2_df

In [112]:
# seg_m1_df.labels.sum()

In [113]:
# seg_m1_df.predicted.sum()

In [114]:
# seg_m2_df.labels.sum()

In [115]:
# seg_m2_df.predicted.sum()

In [116]:
print(classification_report(seg_m1_status_df.labels, seg_m1_status_df.predicted))

              precision    recall  f1-score   support

           0       0.80      0.95      0.86        37
           1       0.95      0.80      0.86        44

    accuracy                           0.86        81
   macro avg       0.87      0.87      0.86        81
weighted avg       0.88      0.86      0.86        81



In [117]:
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(seg_m1_status_df.labels, seg_m1_status_df.predicted).ravel()
(tn, fp, fn, tp)

(35, 2, 9, 35)

In [118]:
seg_m2_df = pd.DataFrame([(seg_m2(p),seg_m2(l)) for (p,l) in segs])
seg_m2_df.columns = ["predicted","labels"]
seg_m2_df

,predicted,labels
0,0,0
1,0,0
2,1,1
3,0,0
4,1,1
...,...,...
76,0,0
77,0,0
78,1,1
79,1,1


In [119]:
print(classification_report(seg_m2_df.labels, seg_m2_df.predicted))

              precision    recall  f1-score   support

           0       0.83      0.92      0.88        38
           1       0.92      0.84      0.88        43

    accuracy                           0.88        81
   macro avg       0.88      0.88      0.88        81
weighted avg       0.88      0.88      0.88        81



In [120]:
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(seg_m2_df.labels, seg_m2_df.predicted).ravel()
(tn, fp, fn, tp)

(35, 3, 7, 36)

In [121]:
to.predicted_vessels[163]

['R PCA']

In [122]:
to.labeled_vessels[163]

['R PCA']

In [123]:
# output_df = raw_df
# output_dir = raw_dir.split(".")[0] + "_AUTOMATED_OCCLUSION_PREDICTIONS.xlsx"
# outputs = {"pred_occlusion":to.pred_occl_statuses,
#            "pred_vessels":to.predicted_vessels,
#            "pred_segments":to.predicted_segments,
#            "volumes":to.get_volumes()
#           }
# for col_name, col_data in outputs.items():
#     output_df[col_name] = col_data

# output_df.to_excel(output_dir,index=False)

In [124]:
to.write_to_excel(raw_df,raw_dir)

In [125]:
pd.read_excel(to.get_output_excel_path()).head()

,Organization,Point of Care,Accession Number,Modality,Exam Code,Exam Description,CPT Code,Report Text,LVO,Side,...,Report Finalized By,Report Finalized Date,pred_occlusion,pred_vessels,pred_segments,true_vessels,true_segments,volumes,unmatched_occlusions,uncertain_occlusions
0,BJH,BJH 0105,55427372,CTA,IMG575,CTA/CTP RAPID STROKE (C),70496,EXAMINATION: Computed tomography angiography (...,0,NaN,...,"Eldaya, Rami W",2020-09-20 17:51:50.000000,0,[],[],[],[],"(0, 0)",[],[]
1,BJH,BJH 0082,55427150,CTA,IMG575,CTA/CTP RAPID STROKE (C),70496,EXAMINATION: Computed tomography angiography (...,0,NaN,...,"Goyal, Manu Shri",2020-09-20 19:32:25.000000,0,[],[],[],[],"(0, 0)",[],[]
2,BJH,BJH 94ICU,55427201,CTA,IMG575,CTA/CTP RAPID STROKE (C),70496,EXAMINATION: Computed tomography angiography (...,0,NaN,...,"Goyal, Manu Shri",2020-09-20 20:04:05.999999,0,[],[],[],[],"(0, 5)",[],[]
3,BJH,BJH 94ICU,55426263,CTA,IMG575,CTA/CTP RAPID STROKE (C),70496,EXAMINATION: Computed tomography angiography (...,1,R/L,...,"Goyal, Manu Shri",2020-09-20 09:56:07.000000,1,['R MCA'],['R M1'],['R MCA'],['R M1'],"(51, 42)",[],[]
4,BJH,BJH 0082,55408274,CTA,IMG575,CTA/CTP RAPID STROKE (C),70496,EXAMINATION: Computed tomography angiography (...,0,NaN,...,"Vo, Katie D.",2020-09-17 07:33:09.000000,0,[],[],[],[],"(0, 0)",[],[]


In [126]:
# bin_pred_vessels = mlb_fit.transform(to.predicted_vessels)
# bin_labeled_vessels = mlb_fit.transform(to.labeled_vessels)
# metrics.confusion_matrix(bin_labeled_vessels.argmax(axis=1), bin_pred_vessels.argmax(axis=1), labels=mlb_fit.classes_)

In [127]:
to.occl_status_diffs

(array([  5,  45,  49,  55,  60,  81,  82, 103, 126, 137, 189, 199, 214,
        219, 224, 233, 270, 334, 337, 350, 351, 363, 402, 413, 416, 428,
        437]),)

In [128]:
to.vessels_diffs

[(),
 (),
 (),
 (),
 (),
 ([], ['R MCA', 'R ICA', 'R CCA']),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 ([], ['L MCA']),
 (),
 (),
 (),
 ([], ['R MCA']),
 (),
 (),
 (),
 (),
 (['R MCA', 'R CCA'], ['R CCA']),
 (['L vertebral'], []),
 (),
 (),
 (),
 (),
 (['R ICA'], []),
 (),
 (),
 (['L vertebral', ' basilar'], ['L vertebral']),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (['R vertebral', ' basilar'], [' basilar']),
 (),
 (),
 (['R MCA'], []),
 ([], ['R MCA']),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (['R vertebral', 'L vertebral'], []),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 ([], ['L PCA']),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 ([], ['R vertebral', 'L PCA']),
 (),
 

In [129]:
to.labeled_vessels

[[],
 [],
 [],
 ['R MCA'],
 [],
 ['R MCA', 'R ICA', 'R CCA'],
 [],
 [],
 [],
 ['R PCA'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['L vertebral'],
 [],
 [],
 ['R ACA'],
 [],
 ['R MCA'],
 [],
 [],
 ['R MCA'],
 [],
 [],
 [],
 [],
 [],
 [],
 ['L MCA'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['R vertebral', 'L vertebral'],
 ['L ICA'],
 ['L MCA'],
 [],
 [],
 [],
 ['R MCA'],
 [],
 [],
 [],
 [],
 ['R CCA'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['L vertebral'],
 [],
 [],
 [],
 [],
 [],
 [],
 ['R MCA'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [' basilar'],
 ['R MCA'],
 [],
 [],
 ['R MCA'],
 [],
 [],
 [],
 ['L MCA', 'L ICA'],
 [],
 [],
 ['R ACA'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['R MCA'],
 [],
 [],
 ['L ICA'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['L PCA'],
 ['R MCA'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['R vertebral', 'L PCA'],
 ['L MCA'],
 [],
 [],
 [],
 [],
 [],
 [],
 ['L MCA', 'L I

In [130]:
to.get_raw_accuracy()

0.8438914027149321

In [131]:
to.get_occlusion_accuracy()

0.9389140271493213

In [132]:
to.get_vessel_accuracy()

0.8891402714932126

In [133]:
to.get_segments_accuracy()

0.9072398190045249

In [134]:
diffs = to.diffs
diffs

[(),
 (),
 (),
 (['R M1'], ['R M1', 'L Vertebral']),
 (),
 ([], ['R CCA', 'R ICA', 'R M1']),
 (),
 (),
 (),
 (['R PCA', 'R P2'], ['R P2']),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (['R ACA'], ['R A2']),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (['L vertebral', 'L V4', 'R V4'], ['R vertebral', 'L vertebral']),
 (),
 ([], ['L M2']),
 (),
 (),
 (),
 ([], ['R M3']),
 (),
 (),
 (),
 (),
 (['R CCA', 'R M1', 'R M2'], ['R CCA']),
 (['L vertebral', 'L V4'], []),
 (),
 (),
 (),
 (),
 (['R ICA'], []),
 (),
 (),
 (['L vertebral', 'basilar', 'L V4'], [' Basilar', 'L V4']),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (['R V4', 'basilar'], ['basilar']),
 (),
 (),
 (['R M1', 'R M2', 'R M3'], []),
 ([], ['R M1']),
 (),
 (),
 (),
 (),
 (),
 (),
 (['R ACA', 'R A2', 'R A2'], ['R A2']),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (['L vertebral', 'L V1', 'R vertebral', 'R V1'], 

In [135]:
to.segments_diffs

[(),
 (),
 (),
 (),
 (),
 ([], ['R M1']),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 ([], ['R A2']),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (['L V4', 'R V4'], []),
 (),
 ([], ['L M2']),
 (),
 (),
 (),
 ([], ['R M3']),
 (),
 (),
 (),
 (),
 (['R M1', 'R M2'], []),
 (['L V4'], []),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (['R V4'], []),
 (),
 (),
 (['R M1', 'R M2', 'R M3'], []),
 ([], ['R M1']),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (['L V1', 'R V1'], []),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 ([], ['L P2']),
 (['R M1', 'R M2'], ['R M2']),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 ([], ['R V3', 'R V4']),
 (['L M1', 'L M2'], ['L M2']),
 (),
 (),
 (),
 (),
 (),
 (),
 (['L M1', 'L M2', 

In [136]:
ind_vessels_diffs = list(filter(lambda xy:True if xy[0] else False,(zip(to.vessels_diffs,range(len(to.vessels_diffs))))))
ind_vessels_diffs

[(([], ['R MCA', 'R ICA', 'R CCA']), 5),
 (([], ['L MCA']), 45),
 (([], ['R MCA']), 49),
 ((['R MCA', 'R CCA'], ['R CCA']), 54),
 ((['L vertebral'], []), 55),
 ((['R ICA'], []), 60),
 ((['L vertebral', ' basilar'], ['L vertebral']), 63),
 ((['R vertebral', ' basilar'], [' basilar']), 78),
 ((['R MCA'], []), 81),
 (([], ['R MCA']), 82),
 ((['R vertebral', 'L vertebral'], []), 103),
 (([], ['L PCA']), 126),
 (([], ['R vertebral', 'L PCA']), 137),
 ((['R PCA', 'R vertebral', 'L vertebral', 'L PCA'], ['R PCA', 'L vertebral']),
  169),
 ((['L MCA', 'R vertebral'], ['L MCA']), 183),
 (([], ['R MCA']), 189),
 (([], ['R MCA']), 199),
 ((['R MCA', 'R ICA'], ['R MCA']), 206),
 ((['L vertebral'], []), 214),
 ((['R vertebral', 'L vertebral', ' basilar'], []), 219),
 ((['R MCA'], []), 224),
 (([], ['R vertebral']), 233),
 (([], ['L ICA']), 270),
 ((['L MCA', 'L vertebral'], ['L MCA']), 271),
 ((['L MCA', 'L ICA', 'L ACA'], ['L ICA']), 280),
 ((['R MCA', 'R vertebral'], ['R MCA']), 283),
 ((['L MCA'

In [137]:
unmatched_occlusion_sents = to.get_unmatched_occlusions()
unmatched_occlusion_sents

[(['  The right vertebral artery is dominant which is severely narrowed/occluded at the origin but reconstitutes distally'],
  43),
 (['  Occlusion of distal right M3/M4 anterior branch within the area of previously noted hypoattenuation/infarct with a paucity of branches noted within the area as compared to the left side with associated area of perfusion abnormality as above in, keeping with diagnosis of distal MCA infarct',
   '  High-grade stenosis/near occlusion of a posterior right distal M3 branch, likely corresponding to the more posterior area of hypoperfusion seen on perfusion imaging'],
  49),
 (['  This may be due to a congenitally diminutive or anomalous vessel origin versus possible occlusion, although there is no associated CT perfusion abnormality'],
  107),
 (['  Findings of subacute infarct involving the left parietal lobe with likely occlusion of the distal left M2 branch or M3 branch'],
  129),
 (['Occlusion of the right M2 segment of the middle cerebral artery with 

In [138]:
uncertain_occlusions = to.get_uncertain_occlusions()
#uncertain_occlusions= [x for x in uncertain_occlusions if x]
uncertain_occlusions

[([('uncertain occlusion',
    'ICA',
    '2 CTA demonstrates nonopacification at the right carotid bifurcation extending into the proximal external carotid artery with partial opacification of the proximal external carotid artery branches with complete nonopacification of the right cervical ICA'),
   ('uncertain occlusion',
    'MCA',
    '  Suboptimal collateral opacification of the carotid terminus on the right is noted with subsequent nonopacification of the right M1 segment and paucity of the distal MCA branches with CTA rapid demonstrating less than 45% contrast density within the right MCA'),
   ('uncertain occlusion',
    'M1',
    '  Suboptimal collateral opacification of the carotid terminus on the right is noted with subsequent nonopacification of the right M1 segment and paucity of the distal MCA branches with CTA rapid demonstrating less than 45% contrast density within the right MCA')],
  5),
 ([('uncertain occlusion',
    'ICA',
    'Nonocclusive filling defect within th

In [139]:
ind_diffs = [ind for sent,ind in ind_vessels_diffs]
ind_unmatched = [ind for sent,ind in unmatched_occlusion_sents]
list(set(np.ndarray.tolist(to.occl_status_diffs[0]))&set(ind_unmatched))

[334, 49, 402, 189, 351]

In [140]:
ind_vessels_diffs
#5 missed occls 419,452, 125, 516, 109
#4 falsely called occl 137, 334, 114, 528

[(([], ['R MCA', 'R ICA', 'R CCA']), 5),
 (([], ['L MCA']), 45),
 (([], ['R MCA']), 49),
 ((['R MCA', 'R CCA'], ['R CCA']), 54),
 ((['L vertebral'], []), 55),
 ((['R ICA'], []), 60),
 ((['L vertebral', ' basilar'], ['L vertebral']), 63),
 ((['R vertebral', ' basilar'], [' basilar']), 78),
 ((['R MCA'], []), 81),
 (([], ['R MCA']), 82),
 ((['R vertebral', 'L vertebral'], []), 103),
 (([], ['L PCA']), 126),
 (([], ['R vertebral', 'L PCA']), 137),
 ((['R PCA', 'R vertebral', 'L vertebral', 'L PCA'], ['R PCA', 'L vertebral']),
  169),
 ((['L MCA', 'R vertebral'], ['L MCA']), 183),
 (([], ['R MCA']), 189),
 (([], ['R MCA']), 199),
 ((['R MCA', 'R ICA'], ['R MCA']), 206),
 ((['L vertebral'], []), 214),
 ((['R vertebral', 'L vertebral', ' basilar'], []), 219),
 ((['R MCA'], []), 224),
 (([], ['R vertebral']), 233),
 (([], ['L ICA']), 270),
 ((['L MCA', 'L vertebral'], ['L MCA']), 271),
 ((['L MCA', 'L ICA', 'L ACA'], ['L ICA']), 280),
 ((['R MCA', 'R vertebral'], ['R MCA']), 283),
 ((['L MCA'

In [141]:
len(unmatched_occlusion_sents)

12

In [142]:
len(ind_vessels_diffs)

49

In [143]:
ind_diffs = [ind for sent,ind in ind_vessels_diffs]
ind_uncertain = [ind for sent,ind in uncertain_occlusions]
list(set(np.ndarray.tolist(to.occl_status_diffs[0]))&set(ind_uncertain))

[224, 5, 199, 81, 126, 60, 189, 350]

In [144]:
len(uncertain_occlusions)

25

In [145]:
#MAKES NO SENSE
#Fix poss ratings
to.possibility_ratings[163]

[('Right distal P1 segment stenosis and/or occlusion', 'L', 'PCA', None),
 ('Right distal P1 segment stenosis and/or occlusion', 'L', 'P1', -1),
 ('Right distal P1 segment stenosis and/or occlusion', 'L', 'P2', None),
 ('Right distal P1 segment stenosis and/or occlusion', 'L', 'P3', None),
 ('Right distal P1 segment stenosis and/or occlusion', 'L', 'P4', None),
 ('Right distal P1 segment stenosis and/or occlusion', 'R', 'PCA', None),
 ('Right distal P1 segment stenosis and/or occlusion', 'R', 'P1', 1),
 ('Right distal P1 segment stenosis and/or occlusion', 'R', 'P2', None),
 ('Right distal P1 segment stenosis and/or occlusion', 'R', 'P3', None),
 ('Right distal P1 segment stenosis and/or occlusion', 'R', 'P4', None)]

In [146]:
markups = list(enumerate(to.markups))
markups[561]

IndexError: list index out of range

In [ ]:
#218
print(filtered_df["Report Text"].iloc[564])

In [ ]:
to.segments_diffs

In [ ]:
to.get_volumes()

In [ ]:
ind_volumes = list(zip(to.get_volumes(),range(len(to.get_volumes()))))
ind_volumes

In [ ]:
indices = [i for tup,i in ind_vessel_diffs if tup]
ratings = [list(zip(ind_vessel_diffs,to.possibility_ratings))[i] for i in indices]
ratings

In [ ]:
predictions = [['L vertebral', 'L ICA', 'L Basilar'],['L MCA']]
labels = [['L ICA', 'L basilar', 'L Vertebral'],['L ICA']]

lower = lambda x:x.lower()
differences = [(prediction,label) if list(set(map(lower,prediction))^set(map(lower,label))) 
               else () for prediction,label in zip(predictions,labels) ]

score = 0
for tup in differences:
    if not tup:
        score += 1
print(score)
sets = [list(set(map(lower,prediction))^set(map(lower,label))) for prediction,label in zip(predictions,labels)]
#list(map(lower,prediction)) != list(map(lower,label))
differences


In [ ]:
"""
Desc:
 Using set() to find the difference between two lists in Python
"""

def list_diff(list1, list2): 
    return (list(set(list1) - set(list2))) 

# Test Input
list1 = [26, 41, 36]
list2 = [11, 16, 21, 26, 31, 36, 41] 


# Run Test
print(list_diff(list1, list2))

In [ ]:
multi = [i for i in range(len(occl_labels)) if isinstance(occl_labels[i],list)]
multi

In [ ]:
multi_occls = [occl_labels[i] for i in multi]
multi_occls

In [ ]:
m_sides = [side_labels[i] for i in multi]
m_sides

In [ ]:
l_sides = [i for i in range(len(side_labels)) if isinstance(side_labels[i],list)]
l_sides

In [ ]:
# import these modules 
# import nltk
# nltk.download('wordnet')
# from nltk.stem import WordNetLemmatizer 
  
# lemmatizer = WordNetLemmatizer() 
  
# print("occlude :", lemmatizer.lemmatize("occlude")) 
#print("corpora :", lemmatizer.lemmatize("corpora")) 

In [ ]:
mid_shift_sents = []
for file in raw_text_files:
    for sent in file.split("."):
        if 'midline shift' in sent:
            mid_shift_sents.append(sent)
mid_shift_sents

In [ ]:
q = [0,-1,2,-3,4,-5]
r = -1
sum(abs(np.multiply(q,r)))

In [ ]:
q = float("nan")
q

In [ ]:
str(q) == "nan"